In [5]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

# importing util.py 
import util

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, isnan, when, count, isnull

In [ ]:
# import sys
# sys.path.append('notebooks/util.py')
# print(sys.path)

In [2]:
data_directory = '../data/airlines/'
util.check_and_fetch_data(data_directory)

CSV files already exist in ../data/airlines/. Skipping data download.


In [3]:
# Uncomment the below code block if the Optimized spark config is not sufficient for your machine 
# spark = SparkSession.builder \
#     .appName("Flight Data Analysis") \
#     .config("spark.sql.debug.maxToStringFields", 1000) \
#     .getOrCreate()

# spark.sparkContext.setLogLevel("ERROR")
# ---------


# keep the following code for the optimized SparkSession builder for our dataset size
# Create a Spark session with optimized configurations
spark = SparkSession.builder \
    .appName("Flight Data Analysis") \
    .config("spark.sql.debug.maxToStringFields", 1000) \
    .config("spark.sql.shuffle.partitions", 1000) \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.cores", "4") \
    .config("spark.sql.autoBroadcastJoinThreshold", "100MB") \
    .getOrCreate()


# Don't comment out the below lines
# Set the log level
spark.sparkContext.setLogLevel("ERROR")

25/04/29 18:49:05 WARN Utils: Your hostname, Mihirs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.7 instead (on interface en0)
25/04/29 18:49:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/29 18:49:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# os.getcwd()
home_dir = os.path.expanduser('~')
folder_path = os.path.join(home_dir, 'Desktop/GroupProject/data/airlines/archive/raw')
# folder_path = '~/Desktop/GroupProject/data/archive/raw'
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))
df = spark.read.csv(csv_files,
                       sep = ',',
                       inferSchema = True,
                       header = True)


In [7]:
num_entries = df.count()
num_entries

29193782

In [8]:
null_counts = df.select([count(col(c)).alias(c) for c in df.columns]).collect()[0].asDict()
print(null_counts)

[Stage 7:======================================================>(115 + 1) / 116]

{'Year': 29193782, 'Quarter': 29193782, 'Month': 29193782, 'DayofMonth': 29193782, 'DayOfWeek': 29193782, 'FlightDate': 29193782, 'Marketing_Airline_Network': 29193782, 'Operated_or_Branded_Code_Share_Partners': 29193782, 'DOT_ID_Marketing_Airline': 29193782, 'IATA_Code_Marketing_Airline': 29193782, 'Flight_Number_Marketing_Airline': 29193782, 'Originally_Scheduled_Code_Share_Airline': 3873, 'DOT_ID_Originally_Scheduled_Code_Share_Airline': 3873, 'IATA_Code_Originally_Scheduled_Code_Share_Airline': 3873, 'Flight_Num_Originally_Scheduled_Code_Share_Airline': 3873, 'Operating_Airline ': 29193782, 'DOT_ID_Operating_Airline': 29193782, 'IATA_Code_Operating_Airline': 29193782, 'Tail_Number': 28926171, 'Flight_Number_Operating_Airline': 29193782, 'OriginAirportID': 29193782, 'OriginAirportSeqID': 29193782, 'OriginCityMarketID': 29193782, 'Origin': 29193782, 'OriginCityName': 29193782, 'OriginState': 29193782, 'OriginStateFips': 29193782, 'OriginStateName': 29193782, 'OriginWac': 29193782, 'D

In [10]:
columns_with_few_nulls = [col_name for col_name, count_val in null_counts.items() if count_val > 0.9*num_entries]
#columns_with_no_nulls = [col_name for col_name, count_val in null_counts.items() if count_val != 0]
print(columns_with_few_nulls)
#print(columns_with_all_nulls)
#df.select(*columns_with_nulls).show()

['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate', 'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners', 'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline', 'Flight_Number_Marketing_Airline', 'Operating_Airline ', 'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline', 'Tail_Number', 'Flight_Number_Operating_Airline', 'OriginAirportID', 'OriginAirportSeqID', 'OriginCityMarketID', 'Origin', 'OriginCityName', 'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac', 'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'Dest', 'DestCityName', 'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'CRSDepTime', 'DepTime', 'DepDelay', 'DepDelayMinutes', 'DepDel15', 'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'CRSArrTime', 'ArrTime', 'ArrDelay', 'ArrDelayMinutes', 'ArrDel15', 'ArrivalDelayGroups', 'ArrTimeBlk', 'Cancelled', 'Diverted', 'CRSElapsedTime', 'ActualElapsedTime', 'AirTime', 'Fligh

In [11]:
newdf = df.select(columns_with_few_nulls)
newdf.select(newdf.columns[0:10]).show()

+----+-------+-----+----------+---------+----------+-------------------------+---------------------------------------+------------------------+---------------------------+
|Year|Quarter|Month|DayofMonth|DayOfWeek|FlightDate|Marketing_Airline_Network|Operated_or_Branded_Code_Share_Partners|DOT_ID_Marketing_Airline|IATA_Code_Marketing_Airline|
+----+-------+-----+----------+---------+----------+-------------------------+---------------------------------------+------------------------+---------------------------+
|2020|      4|   10|        15|        4|2020-10-15|                       UA|                           UA_CODESHARE|                   19977|                         UA|
|2020|      4|   10|        23|        5|2020-10-23|                       UA|                           UA_CODESHARE|                   19977|                         UA|
|2020|      4|   10|        20|        2|2020-10-20|                       UA|                           UA_CODESHARE|                   199

In [ ]:
newdf.select(newdf.columns[10:19]).show()

In [ ]:
newdf.select(newdf.columns[19:33]).show()

In [ ]:
newdf.select(newdf.columns[33:50]).show()

In [ ]:
newdf.select(newdf.columns[50:62]).show()

In [13]:
repeat_cols = ["Marketing_Airline_Network", "Operated_or_Branded_Code_Share_Partners", "IATA_Code_Marketing_Airline", "Operating_Airline ", "IATA_Code_Operating_Airline"]
print(newdf.select([count(col(c)).alias(c) for c in repeat_cols]).collect()[0].asDict())

[Stage 12:=================================================>    (107 + 9) / 116]

{'Marketing_Airline_Network': 29193782, 'Operated_or_Branded_Code_Share_Partners': 29193782, 'IATA_Code_Marketing_Airline': 29193782, 'Operating_Airline ': 29193782, 'IATA_Code_Operating_Airline': 29193782}


In [14]:
print(newdf.select(repeat_cols).distinct().count())

[Stage 15:=====================================================>(115 + 1) / 116]

39


In [12]:
cols_to_keep = ["Year", "Month", "DayofMonth", "Origin", "OriginCityName", "DestCityName", "DepDelay", "ArrDelay", "Cancelled", "CRSElapsedTime", "ActualElapsedTime"]
my_df = newdf.select(cols_to_keep)
my_df.show()

+----+-----+----------+------+--------------------+--------------------+--------+--------+---------+--------------+-----------------+
|Year|Month|DayofMonth|Origin|      OriginCityName|        DestCityName|DepDelay|ArrDelay|Cancelled|CRSElapsedTime|ActualElapsedTime|
+----+-----+----------+------+--------------------+--------------------+--------+--------+---------+--------------+-----------------+
|2020|   10|        15|   CLT|       Charlotte, NC|      Washington, DC|    -1.0|    19.0|      0.0|          96.0|            116.0|
|2020|   10|        23|   PIT|      Pittsburgh, PA|         Houston, TX|    -4.0|    16.0|      0.0|         193.0|            213.0|
|2020|   10|        20|   SDF|      Louisville, KY|         Houston, TX|     0.0|    32.0|      0.0|         150.0|            182.0|
|2020|   10|        20|   LIT|     Little Rock, AR|         Houston, TX|    -1.0|    29.0|      0.0|          99.0|            129.0|
|2020|   10|        17|   IAD|      Washington, DC|Sarasota/Br

## Which Origin Cities had the most delayed flights?

In [ ]:
count_delay = my_df.select(["Origin", "DepDelay"]).groupBy("Origin")\
        .agg(count(F.when(col("DepDelay") > 0, 1)).alias("DelayCount"), 
             count(F.when(col("DepDelay") < 0, 1)).alias("EarlyCount"),
            count("*").alias("TotalCount")).orderBy(col("TotalCount").desc())
pandas_delay = count_delay.toPandas()

In [ ]:
pdf = pandas_delay.copy()
pdf["OnTimeCount"] = pdf["TotalCount"] - pdf["DelayCount"] - pdf["EarlyCount"]

In [ ]:
top_20 = pdf.head(20)
top_20

In [ ]:
count_delay = my_df.select(["Origin", "DepDelay"]).groupBy("Origin")\
        .agg(count(F.when(col("DepDelay") > 0, 1)).alias("DelayCount"), 
             count(F.when(col("DepDelay") < 0, 1)).alias("EarlyCount"),
            count("*").alias("TotalCount")).orderBy(col("TotalCount").desc())
pandas_delay = count_delay.toPandas()# Assuming pdf has these columns: OriginCity, DelayedFlights, EarlyFlights, OnTimeFlights

# Bar positions
cities = top_20["Origin"]
x = np.arange(len(cities))

# Heights
early = top_20["EarlyCount"]
on_time = top_20["OnTimeCount"]
delayed = top_20["DelayCount"]

# Plot
plt.figure(figsize=(12, 6))
plt.bar(x, early, label="Early", color="green")
plt.bar(x, on_time, bottom=early, label="On Time", color="gray")
plt.bar(x, delayed, bottom=early + on_time, label="Delayed", color="red")

# Labels and formatting
plt.xticks(x, cities, rotation=45)
plt.ylabel("Number of Flights")
plt.title("Flight Status by Origin City (Top 20)")
plt.legend(title="Flight Status")
plt.tight_layout()
plt.show()

In [ ]:
year_delay = my_df.select(["Year", "DepDelay"]).groupBy("Origin")\
        .agg(count(F.when(col("DepDelay") > 0, 1)).alias("DelayCount"), 
             count(F.when(col("DepDelay") < 0, 1)).alias("EarlyCount"),
            count("*").alias("TotalCount")).orderBy(col("TotalCount").desc())
pandas_year_delay = count_delay.toPandas()